In [27]:
fn parse_input(inp: &str) -> Vec<(&str, &str)> {
    inp.split('\n').map(|line| {
        let mut parts = line.split('-').take(2);
        (parts.next().unwrap(), parts.next().unwrap())
    }).collect()
}

In [28]:
let inp = parse_input("nu-start
rt-start
db-qh
PE-end
sl-rt
qh-end
ZH-rt
nu-rt
PE-db
db-sl
nu-ZH
nu-qh
PE-qh
ZH-db
ne-end
ne-ZH
QG-db
qh-sl
ZH-qh
start-ZH
nu-PE
uf-db
ne-sl");
inp

[("nu", "start"), ("rt", "start"), ("db", "qh"), ("PE", "end"), ("sl", "rt"), ("qh", "end"), ("ZH", "rt"), ("nu", "rt"), ("PE", "db"), ("db", "sl"), ("nu", "ZH"), ("nu", "qh"), ("PE", "qh"), ("ZH", "db"), ("ne", "end"), ("ne", "ZH"), ("QG", "db"), ("qh", "sl"), ("ZH", "qh"), ("start", "ZH"), ("nu", "PE"), ("uf", "db"), ("ne", "sl")]

In [29]:
use std::collections::{HashMap, HashSet, VecDeque};

#[derive(Clone, Default, Debug)]
struct Path<'a> {
    path: Vec<&'a str>,
    visited_small: HashSet<&'a str>,
    visited_twice: bool,
}

fn is_small(elem: &str) -> bool {
    elem.chars().next().map(char::is_lowercase).unwrap_or(false)
}

impl<'a> Path<'a> {
    fn after_visiting(&self, elem: &'a str, allow_twice: bool) -> Option<Self> {
        let already_visited = is_small(elem) && self.visited_small.contains(elem);
        if !already_visited || (allow_twice && already_visited && !self.visited_twice) {
            let mut path = self.path.clone();
            path.push(elem);
            let visited_twice = self.visited_twice || self.visited_small.contains(elem);
            let mut visited_small = self.visited_small.clone();
            if is_small(elem) {
                visited_small.insert(elem);
            }
            Some(Self { path, visited_small, visited_twice })
        } else {
            None
        }
    }
}

In [30]:
fn solve<'a>(pairs: &[(&'a str, &'a str)], allow_twice: bool) -> Vec<Path<'a>> {
    let mut full_paths = Vec::new();
    
    let mut neighbours: HashMap<&'a str, Vec<&'a str>> = Default::default();
    for (a, b) in pairs {
        if a != &"end" && b != &"start" {
            neighbours.entry(a).or_default().push(b);
        }
        if b != &"end" && a != &"start" {
            neighbours.entry(b).or_default().push(a);
        }
    }

    let mut queue: VecDeque<Path<'a>> = Default::default();
    queue.push_back(Path::default().after_visiting("start", false).unwrap());
    while let Some(path) = queue.pop_front() {
        for neighbour in neighbours.get(path.path.last().unwrap()).unwrap_or(&vec![]) {
            if let Some(new_path) = path.after_visiting(neighbour, allow_twice) {
                if new_path.path.last().unwrap() == &"end" {
                    full_paths.push(new_path);
                } else {
                    queue.push_back(new_path);
                }
            }
        }
    }

    full_paths
}

In [31]:
let sample1 = parse_input("start-A
start-b
A-c
A-b
b-d
A-end
b-end");
let sample2 = parse_input("dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc");
let sample3 = parse_input("fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW");

for i in [&sample1, &sample2, &sample3] {
    dbg!(solve(i, false).len());
}

[src/lib.rs:232] solve(i, false).len() = 10
[src/lib.rs:232] solve(i, false).len() = 19
[src/lib.rs:232] solve(i, false).len() = 226


()

In [32]:
solve(&inp, false).len()

4338

In [33]:
for i in [&sample1, &sample2, &sample3] {
    dbg!(solve(i, true).len());
}

[src/lib.rs:196] solve(i, true).len() = 36
[src/lib.rs:196] solve(i, true).len() = 103
[src/lib.rs:196] solve(i, true).len() = 3509


()

In [34]:
solve(&inp, true).len()

114189